In [18]:
import numpy as np

In [19]:
def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

In [20]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f'{type(data)}은(는) 지원하지 않습니다.')

        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop() # 함수를 가져온다.

            gys = [output.grad for output in f.outputs] # ❶
            gxs = f.backward(*gys) # ❷
            if not isinstance(gxs, tuple): # ❸
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs): #❹
                x.grad = gx

                if x.creator is not None:
                    funcs.append(x.creator)

In [21]:
class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)        # ❶ 별표를 붙여 언팩
        if not isinstance(ys, tuple): # ❷ 튜플이 아닌 경우 추가 지원
            ys = (ys,)

        outputs = [Variable(as_array(y)) for y in ys]  # Variable 형태로 되돌린다.
        for output in outputs:
            output.set_creator(self) # 출력 변수에 창조자를 설정한다.

        self.inputs  = inputs       # 입력 변수를 기억(보관)한다.
        self.outputs = outputs      # 출력도 저장한다.

        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()

In [22]:
class Square(Function):
    def forward(self, x):
        return x ** 2

    def backward(self, gy):
        x = self.inputs[0].data
        gx = 2 * x * gy
        return gx

class Exp(Function):
    def forward(self, x):
        return np.exp(x)

    def backward(self, gy):
        x = self.inputs[0].data
        gx = np.exp(x) * gy
        return gx

class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y

    def backward(self, gy):
        return gy, gy

In [23]:
def square(x):
    f = Square()
    return f(x)

def exp(x):
    f = Exp()
    return f(x)

def add(x0, x1):
    f = Add()
    return f(x0, x1)

In [24]:
x0 = Variable(np.array(2))
x1 = Variable(np.array(3))
y = add(x0, x1) # Add 클래스 생성 과정이 감춰짐
print(y.data)

5


In [25]:
x = Variable(np.array(0.5))
y = square(exp(square(x))) # 연속하여 적용
print(y.data)
y.backward()
print(x.grad)

1.648721270700128
3.297442541400256


In [27]:
x = Variable(np.array(2.0))
y = Variable(np.array(3.0))

z = add(square(x), square(y))
z.backward()

print(z.data)
print(z.grad)
print(x.grad)
print(y.grad)

13.0
1.0
4.0
6.0
